In [4]:
import chainlit as cl
import os
from llama_index.core import Settings, VectorStoreIndex, StorageContext, SimpleDirectoryReader, Document
from llama_index.llms.together import TogetherLLM
from llama_index.embeddings.together import TogetherEmbedding
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core import PromptTemplate

llm = TogetherLLM(
    model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo", api_key="dae9b33beef8cc57bc9475d53519e9e70037e6820b8c2d54b0b2f349e11101b0"
)

embed_model = TogetherEmbedding(
    model_name = "togethercomputer/m2-bert-80M-8k-retrieval", api_key="dae9b33beef8cc57bc9475d53519e9e70037e6820b8c2d54b0b2f349e11101b0"
)

In [8]:
documents = SimpleDirectoryReader(input_dir="./data", recursive=True).load_data()
vector_store = MilvusVectorStore(uri="./milvus_demo.db", dim=768, overwrite=True)

2024-07-21 05:43:41 - Created new connection using: 3e6ac38bdc5541b980f766f7fff8853a
2024-07-21 05:43:41 - Successfully created collection: llamacollection
2024-07-21 05:43:41 - Successfully created an index on collection: llamacollection


In [9]:
# local_model_endpoint = "http://localhost:8000"
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# index = VectorStoreIndex.from_documents(embed_model=embed_model,documents=documents,storage_context=storage_context)

In [10]:
Settings.embed_model = embed_model
Settings.llm = llm

In [11]:

index = VectorStoreIndex.from_documents(documents=documents)
query_engine = index.as_query_engine()
result = query_engine.query("Can you give me some names of common fever medications?")

2024-07-21 05:44:00 - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"


In [12]:
print(result)

Acetaminophen, ibuprofen, and aspirin are some common fever medications.


In [13]:
promptTemplate = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Given this information, please answer the question: {query_str}\n"
    "If the question is out of the scope of the given information, answer to the best of your ability"
)

In [14]:
qa_template = PromptTemplate(promptTemplate)

In [15]:
question = "What medications should I take for a urinary tract information?"
prompt = qa_template.format(context_str = documents, query_str = question)
response = llm.complete(prompt)
print(response.text)

2024-07-21 05:44:13 - HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
Based on the provided information, the answer to the question "What medications should I take for a urinary tract infection?" is that acetaminophen or ibuprofen can help relieve the pain associated with urinary tract infections. However, it's important to note that these medications do not cure infections, and you should see a doctor for proper treatment.


In [16]:
from llama_index.core import StorageContext
from llama_index.core import load_index_from_storage

In [19]:
index.storage_context.persist("medicine_index")
storage_context = StorageContext.from_defaults(persist_dir="medicine_index")
new_index = load_index_from_storage(storage_context)

2024-07-21 05:44:56 - Loading all indices.
